<a href="https://colab.research.google.com/github/morgwork/Distantly-Reading-IPCC-Reports/blob/main/IPCC_Project_Impacts6_(4_14).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This codebook converts, cleans, and analyses the Sixth Assessment cycle "Impacts" report. The PDF is 3,676 pages. It is available at this link: https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_FinalDraft_FullReport.pdf. The cover page says "Summary for Policymakers" but, on the IPCC website, that is what links to "Full Report."**


### Conversion
In this section I convert the PDF report to a machine-readble string. Most of this code is from an external library, so I put links in as reference. The code's ultimate output is a string for subsequent cleaning and analysis.

In [ ]:
# opens PDF. I suppose this is just to make sure it can be read...realize now that it's separate libraries

#install pyDF2
! pip install PyPDF2

# importing all the required modules
import PyPDF2

# creating an object 
file = open('/content/IPCC6I.pdf', 'rb') # put file here 

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)

# print the number of pages in pdf file
print(fileReader.numPages) # fileReader becomes object

     |████████████████████████████████| 76 kB 2.8 MB/s 
3676


In [ ]:
! pip install pdfminer # successful installation of PDFminer
import pdfminer

     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 2.0 MB 38.8 MB/s 
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140093 sha256=cba933e16158ceed975db4ffa74acc55562ca3ff9a3ed3bbd84718381216292e
  Stored in directory: /root/.cache/pip/wheels/e3/5e/f4/d210b46e9e4a28229ea070ed5b3efa92c3c29d1a7918dd4b97
Successfully built pdfminer


Got the following code from: https://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library/8325135#8325135. 

In [ ]:
from io import StringIO

Learned about this from: https://stackoverflow.com/questions/28200366/python-3-x-importerror-no-module-named-cstringio.

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams(char_margin = 20) 
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open('/content/IPCC6I.pdf', 'rb') # have to put file path in here
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [ ]:
# here is where we get a string from the PDF, now need to export the string as txt
convert_pdf_to_txt(fileReader)
Impacts_6=convert_pdf_to_txt(fileReader) # decide on new object name for converted file at the start here
print(Impacts_6) # object to make sure it comes out as a string. this works! just takes a little bit of time

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Cleaning
In this section, I  case, lemm, and convert the cleaned files for analysis. 

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [40]:
def preprocess_text(text: str, remove_stopwords: bool) -> str: #creating a function with the if and below
#just have to define ahead of time, otherwise unable to use as code in other codebooks
    """This utility function sanitizes a string by: 
    - removing links
    - removing special characters
    - removing numbers
    - removing stopwords
    - transforming in lowercase
    - removing excessive whitespaces
    Args:
        text (str): the input text you want to clean
        remove_stopwords (bool): whether or not to remove stopwords
    Returns:
        str: the cleaned text
    """ #three quote marks designate function's documentation

    # remove links
    text = re.sub(r"http\S+", "", text) #re is the regular expression library from above
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text) #notice the white space between quotes preceding "text" in each line
    # remove stopwords
    if remove_stopwords:
        # 1. tokenize
        tokens = nltk.word_tokenize(text)
        # 2. check if stopword
        tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
        # 3. join back together
        text = " ".join(tokens)
    # return text in lower case and stripped of whitespaces
    text = text.lower().strip()
    return text

In [30]:
with open("/content/IPCC6M.txt", "r", encoding='utf-8') as text:
    IPCCM_6 = text.read()

In [47]:
import string

all_punctuation = string.punctuation

print(f"All punctuation: {all_punctuation}")

All punctuation: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [65]:
# Remove stopwords function.

def remove_Stopwords(text):
    stop_words = set(stopwords.words('english')) 
    words = word_tokenize( text.lower() ) 
    sentence = [w for w in words if not w in stop_words]
    return " ".join(sentence)

# Lemmatize function.    
def lemmatize_text(text):
    wordlist=[]
    lemmatizer = WordNetLemmatizer() 
    sentences=sent_tokenize(text)
    for sentence in sentences:
        words=word_tokenize(sentence)
        for word in words:
            wordlist.append(lemmatizer.lemmatize(word))
    return ' '.join(wordlist) 

# Cleaning text function.
def clean_text(text): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr]) 
    
    return text2.lower()


In [53]:
MC1= IPCCM_6.lower() # remove casing
MC2=MC1
print(MC2[:500])

﻿ipcc
intergovernmental panel on cllffldto olldago
climate change 2022
mitigation of climate change
summary for policymakers
working group iii contribution to the sixth assessment report of the intergovernmental panel on climate change
wmo unep
approved
summary for policymakers
ipcc ar6 wg iii
working group iii contribution to the ipcc sixth assessment report (ar6)
summary for policymakers
drafting authors:
jim skea (united kingdom), priyadarshi r shukla (india), andy reisinger (new zealand), ra


In [54]:
for p in all_punctuation:

  MC3=MC2.replace(p,'')

In [55]:
IPCCM6_words= MC3.split()
print('Number of words in IPCCM6:',len(IPCCM6_words))

Number of words in IPCCM6: 1577262


In [62]:
MC4=remove_Stopwords(MC3)

In [63]:
MC5=lemmatize_text(MC4)

In [66]:
MC6=clean_text(MC5)

In [ ]:
# need to append MC6 to a dataframe, I think, to do more with it beyond this cleaning

In [69]:
print(MC6[:2000])

﻿ipcc intergovernmental panel cllffldto olldago climate change 2022 mitigation climate change summary policymakers working group iii contribution sixth assessment report intergovernmental panel climate change wmo unep approved summary policymakers ipcc ar6 wg iii working group iii contribution ipcc sixth assessment report ar6 summary policymakers drafting author jim skea united kingdom priyadarshi r shukla india andy reisinger new zealand raphael slade united kingdom minal pathak india alaa al khourdajie united kingdomsyria renee van diemen netherlandsunited kingdom amjad abdulla maldives keigo akimoto japan mustafa babiker sudansaudi arabia quan bai china igor bashmakov russian federation christopher bataille canada goran berndes sweden gabriel blanco argentina kornelis blok netherlands mercedes bustamante brazil edward byers austria luisa f cabeza spain katherine calvin united state america carlo carraro italy leon clarke united state america annette cowie australia felix creutzig ge

## Analysis
In this section, I will analyze the cleaned files with some basic word counts to highlight significant terms from each of the reports. I will also create a few WordCloud visualizations for each report, one with all collocates true and  two or three that remove disproportionately represented terms and bigrams.

In [70]:
from collections import Counter

words = MC6.split() # need to assign object to function, from week 4 codebook

types = Counter(words)

print(types) # may not be able to remove numbers, will have to see what comes with other reports. 

Counter({'al': 18238, 'et': 18204, 'energy': 11912, 'emission': 9907, 'climate': 9325, 'policy': 7603, '2019': 7570, 'j': 6837, '2018': 6687, '2020': 6545, 'change': 6427, 'mitigation': 6279, 'http': 5432, '1': 5294, 's': 5108, 'm': 4880, 'carbon': 4805, '2': 4737, 'chapter': 4695, 'global': 4660, '2017': 4646, 'a': 4577, '3': 4551, '4': 4282, '5': 4128, '10': 3939, '6': 3896, '2021': 3879, '12': 3826, 'development': 3763, 'total': 3702, 'ipcc': 3693, '8': 3679, '15': 3648, '7': 3625, 'system': 3587, 'government': 3546, '11': 3545, '2016': 3542, 'country': 3502, '9': 3489, '13': 3447, 'technology': 3424, '16': 3272, 'ar6': 3266, '14': 3237, 'c': 3229, 'final': 3120, '20': 3069, '17': 3051, '18': 2998, 'impact': 2953, 'page': 2948, 'sector': 2932, '2015': 2929, '19': 2877, 'distribute': 2854, 'cite': 2850, 'quote': 2847, '25': 2834, 'distribution': 2828, '21': 2814, 'r': 2806, 'wgiii': 2805, '22': 2780, '23': 2774, '24': 2725, 'potential': 2700, '30': 2698, 'high': 2653, '26': 2636, '27

In [71]:
import numpy as np # from bibliometric analysis codebook
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from wordcloud import WordCloud

In [ ]:
df=pd.DataFrame({'author': ['IPCC'], 'report': MC6}) # quite big with interactive view
df

In [ ]:
# keep thinking about what to do in terms of analysis here. got a mostly clean file!

In [74]:
wordcloud = WordCloud(background_color=(246,235,189),max_words=10, width=1600, height=800,random_state=30,stopwords=[]).generate(' '.join(df['types'].tolist())) # need to get reports into a dataframe of some kind
plt.figure( figsize=(20,10))
plt.imshow(wordcloud);

NameError: ignored